In [13]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON

class HealthDM(object):
    profile = None
    profilePresent = None
    profileAbsent = None
    
    sparqlQuery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>

SELECT ?d
FROM <http://id.nlm.nih.gov/mesh>
WHERE {{
    ?d a meshv:Term .
    {{?d meshv:prefLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
    UNION
    {{?d meshv:altLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
}}
ORDER BY ?d
"""
    
    # @classmethod
    # def setProfile(cls, profile):
    #     cls.profile = profile
    
    @classmethod
    def createProfile(cls):
        cls.profile = widgets.Accordion(children=[])
        return cls.profile
        
    @classmethod
    def clearTerms(cls):
        cls.profilePresent = widgets.Accordion(children=[])
        cls.profileAbsent = widgets.Accordion(children=[])
        cls.profile.children = tuple([cls.profilePresent, cls.profileAbsent])
        cls.profile.set_title(0, "Sign Present")
        cls.profile.set_title(1, "Sign Absent")

    @classmethod
    def addTerm(cls, heading, description, code, present, detail1, rate1, detail2, rate2):
        wprofile = [widgets.Text(description = "MeSH Term:", value = heading),
                    widgets.Text(description = "MeSH Code:", value = code)]
        if detail1 != "#":
            fDetail = cls.formatDetail(detail1, rate1)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        if detail2 != "#":
            fDetail = cls.formatDetail(detail2, rate2)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        wcontent = widgets.VBox(wprofile)
        profileList = cls.profilePresent if present else cls.profileAbsent
        profileList.children = tuple(list(profileList.children) + [wcontent])
        profileList.set_title(len(profileList.children)-1, description)
        
    @classmethod
    def formatDetail(cls, detail, rate):
        detailDesc = "Detail:"
        detailValue = detail
        if detail in ["occasionally", "often", "frequently", "sometimes",
                      "rarely", "daily", "monthly", "yearly", "each day",
                      "each month", "each year", "once a day", "once a month",
                      "once a year"]:
            detailDesc = "Frequency:"
        else:
            try:
                value = int(detail)
                detailDesc = "Intensity:"
                if rate != "#":
                    detailDesc = "Frequency:"
                    detailValue = detailValue + " / " + rate
            except ValueError:
                pass
        return [detailDesc, detailValue]

    @classmethod
    def findMeshCode(cls, heading, detail1, rate1, description, detail2, rate2):
        code = "none"
        present = True
        meshHeading = heading
        
        sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
        sparql.setReturnFormat(JSON)

        # looking for the heading
        sparql.setQuery(cls.sparqlQuery.format(meshHeading))
        results = sparql.query().convert()
        if len(results["results"]["bindings"]) > 0:
            code = results["results"]["bindings"][0]["d"]["value"]

        if code == "none" and (heading.lower().startswith("no") or
                               heading.lower().startswith("not")):
            # looking for the heading
            present = False
            meshHeading = heading[3:] if heading.lower().startswith("no") else heading[4:]
            sparql.setQuery(cls.sparqlQuery.format(meshHeading))
            results = sparql.query().convert()
            if len(results["results"]["bindings"]) > 0:
                code = results["results"]["bindings"][0]["d"]["value"]
            
        cls.addTerm(meshHeading, description, code, present, detail1, rate1, detail2, rate2)
    
        return "#mesh_heading#" + heading + "#tree_number#" + code
    
    @classmethod
    def interfacePresentation(cls, title, description, template):
        indexTemplate = open("template/index.html", "r", encoding="utf-8")
        presentationTemplate = open("template/Presentation.html", "r", encoding="utf-8")

        indexResult = open("html/index.html", "w", encoding="utf-8")
        presentationResult = open("html/Presentation.html", "w", encoding="utf-8")
        
        indexResult.write(indexTemplate.read().format(title=title))
        presentationResult.write(presentationTemplate.read().format(title=title, description=description))

        indexResult.close()
        presentationResult.close()

    @classmethod
    def interfaceKnot(cls, htmlName, title, description, template):
        knotTemplate = open("template/" + template + ".html", "r", encoding="utf-8")

        knotResult = open("html/" + htmlName + ".html", "w", encoding="utf-8")
        
        knotResult.write(knotTemplate.read().format(title=title, description=description))

        knotResult.close()

HealthDM.createProfile()

Accordion()

=== Zombie Health ===
Você é um Médico Girafa (girafas não viram zumbis).
=== Lingua vermelha (notice) ===
![Red_tongue](images/tongue-red.png)
A língua está vermelha.
=== Lingua amarela (notice) ===
![Yellow_tongue](images/tongue-yellow.png)
A língua está amarela.
=== Sem Paralisia (notice) ===
![No_paralysis](images/paralysis-no.png)
O está com bons movimentos.
=== Paralisia (notice) ===
![Paralysis](images/paralysis.png)
O corpo está paralisado.
=== Dedos ok (notice) ===
![Fingers_ok](images/fingers-ok.png)
A mãos não têm nenhuma irregularidade observável.
=== Dedos tremendo (notice) ===
![Fingers_shaking](images/fingers-shaking.png)
Os dedos mindinhos estão tremendo.
=== Membros ok (notice) ===
![Members_ok](images/members-ok.png)
Todos os membros estão no lugar.
=== Perda membros (notice) ===
![Members_loss](images/members-loss.png)
Foram perdidos alguns membros.
=== Sem dor (notice) ===
![Pain_no](images/pain-no.png)
Não tem nenhuma queixa de dor.
=== Dor no peito (notice) ===
![Pain_chest](images/pain-chest.png)
Sente uma forte dor no peito.
=== Sem raiva (notice) ===
![Pain_no](images/anger-no.png)
Está de bom humor apesar de tudo.
=== Com raiva (notice) ===
![Pain_chest](images/anger.png)
Sente uma grande raiva.
=== Bacterias ok (notice right) ===
Parabéns você tratou corretamente uma infecção por bactérias.
=== Bacterias errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por bactérias.
=== Virus ok (notice right) ===
Parabéns você tratou corretamente uma infecção por vírus.
=== Virus errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por vírus.
=== Zulumbriga ok (notice right) ===
Parabéns você tratou corretamente uma infecção por zulumbriga.
=== Zulumbriga errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por zulumbriga.
=== Briga ok (notice right) ===
Parabéns você tratou corretamente problemas que surgiram por uma briga.
=== Briga errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, eram problemas que surgiram por uma briga.
=== Caso 1 (case) ===
![Rot_Donnadd](images/1.png)
Rot Donnadd se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Examinar língua -> Lingua amarela
++ Ver movimentos -> Paralisia
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Sem dor
++ Ver humor -> Sem raiva
++ Aplicar tratamento -> Tratamento 1
=== Tratamento 1 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Trata Bactérias -> Bacterias ok
++ Trata Vírus -> Bacterias errado
++ Trata Zulumbriga -> Bacterias errado
++ Trata Brigas -> Bacterias errado
=== Caso 2 ===
Segundo Caso.

In [14]:
%%HTML
<iframe width="100%" height="800px" src="html/index.html"><iframe>